In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv('emotions.txt',names=['sentence','emotions'], sep=';')
df

In [ ]:
label_count = df['emotions'].value_counts()

for i in range(0,6):
    print(f'Class {i} : {label_count[i]}')

label_count.plot(kind='bar', title='Count label')
plt.show()

In [ ]:
df_class_0 = df[df['emotions'] == 'joy']
df_class_1 = df[df['emotions'] == 'sadness']
df_class_2 = df[df['emotions'] == 'anger']
df_class_3 = df[df['emotions'] == 'fear']
df_class_4 = df[df['emotions'] == 'love']
df_class_5 = df[df['emotions'] == 'surprise']

df_class_1_over = df_class_1.sample(label_count[0], replace=True)
df_class_2_over = df_class_2.sample(label_count[0], replace=True)
df_class_3_over = df_class_3.sample(label_count[0], replace=True)
df_class_4_over = df_class_4.sample(label_count[0], replace=True)
df_class_5_over = df_class_5.sample(label_count[0], replace=True)

df_over = pd.concat([
                    df_class_0,
                    df_class_1_over,
                    df_class_2_over,
                    df_class_3_over,
                    df_class_4_over,
                    df_class_5_over],axis=0)

print('Random Over Sampling')
print(df_over['emotions'].value_counts())
df_over['emotions'].value_counts().plot(kind='bar', title='Count label')
plt.show()

In [ ]:
label = pd.get_dummies(df_over['emotions'])
datasets = pd.concat([df_over, label], axis=1)
datasets = datasets.drop(columns='emotions')
datasets

In [ ]:
label_name = [
    'anger',
    'fear',
    'joy',
    'love',
    'sadness',
    'surprise',
    ]

sentence = datasets['sentence'].values
label = datasets[label_name].values

train_tweet, test_tweet, train_label, test_label = train_test_split(
    sentence, 
    label, 
    test_size=0.2,
    random_state=1,
    )

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(train_tweet)
tokenizer.fit_on_texts(test_tweet)

train_sequence = tokenizer.texts_to_sequences(train_tweet)
test_sequence = tokenizer.texts_to_sequences(test_tweet)

train_padded = pad_sequences(train_sequence)
test_padded = pad_sequences(test_sequence)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')>0.95):
            self.model.stop_training = True;
            print('\nAkurasi telah melewati 95%')

callbacks = myCallback()

In [ ]:
model = Sequential([
    layers.Embedding(input_dim=5000, output_dim=16),
    layers.LSTM(64),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax'),
    ])
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )
hist = model.fit(
    train_padded,
    train_label, 
    epochs=30,
    validation_data=(test_padded, test_label),
    verbose=2,
    callbacks=[callbacks],
    )

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')

plt.show()